In [26]:
import sys
sys.path.insert(0, "../..")
from src.data import data_tools
from pathlib import Path
import gin
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from src.models import rnn_models, train_model

datadir = Path("../../data/raw")

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 

# 1.1 Iterator design pattern
This is a nice opportunity to create our own custom iterator. First, let's look at the simplified design pattern for an iterator.

In [27]:
class BaseIterator:
  def __init__(self, n: int):
    self.n = n
    self.data = [*range(self.n)]

  def __iter__(self):
    self.idx = -1
    return self

  def __next__(self):
    if self.idx < len(self.data) - 1:
      self.idx += 1
      return self.data[self.idx]
    else:
      raise StopIteration

myclass = BaseIterator(n=5)
myiter = iter(myclass)

for x in myiter:
  print(x)

0
1
2
3
4


Note how `__iter__` returns `self` with the startsettings (`idx=-1`).
Every time we call `__next__`, e.g. in a forloop or explicitly with `next()` 

If you keep calling `next()`, you will get a `StopIteration` error but the forloop will look out for that, so you should be good.

## 1.2 Iterator implementation
Now, we extend this pattern to our dataset. We will use what we have seen before with the images: we load the paths, and use these to generate the data.

In [28]:
import random
from tqdm import tqdm
class Datagenerator:
    def __init__(self, paths: List[Path], batchsize:int ):
        self.paths = paths
        random.shuffle(self.paths)
        self.batchsize = batchsize

        self.dataset = []
        # If the files are too much to load at once,
        # you could move this to __getitem__
        # however, this is a small dataset and it is faster to load from memory
        for file in tqdm(self.paths):
            x = np.genfromtxt(file)[:, 3:]
            x = torch.tensor(x).type(torch.float32)
            y = int(file.parent.name) - 1
            self.dataset.append((x, y))

        self.size = len(self.dataset)

    
    def __len__(self):
        return int(len(self.dataset) / self.batchsize)
    
    def __getitem__(self, idx):
        return self.dataset[idx]
    
    def __iter__(self):
        self.index = 0
        self.index_list = torch.randperm(self.size) 
        return self

    def __next__(self):
        if self.index <= (len(self.dataset) - self.batchsize):
            X = []
            Y = []
            for _ in range(self.batchsize):
                x, y = self[self.index_list[self.index]]
                X.append(x)
                Y.append(y)
                self.index += 1
            # this makes all sequence of equal length by adding zeros
            X_ = pad_sequence(X, batch_first=True, padding_value=0)
            return X_, torch.tensor(Y)
        else:
            raise StopIteration

The main difference with the pattern we used before, is that this pattern will stop after the last item is spit out.
The generator from the first lesson will go on forever, due to the `while True` loop.

## 1.3 Train-test

In [29]:
data_dir = Path("../../data/external/gestures-dataset/")

# get all paths with the .txt extension
formats = [".txt"]
paths = [path for path in data_tools.walk_dir(data_dir) if path.suffix in formats]
# make a train-test split
split = 0.8
idx = int(len(paths) * split)
trainpaths = paths[:idx]
testpaths = paths[idx:]

In [30]:
trainloader = Datagenerator(trainpaths, batchsize=32)
testloader = Datagenerator(testpaths, batchsize=32)

100%|██████████| 651/651 [00:00<00:00, 1371.31it/s]


In [31]:
x, y = trainloader[0]
x.shape, y

(torch.Size([18, 3]), 8)

In [32]:
len(trainloader)

81

What does the lenght mean?

In [33]:
x, y = next(iter(trainloader))
x.shape, y.shape

(torch.Size([32, 27, 3]), torch.Size([32]))

Can you make sense of the shape?

# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [36]:
gin.parse_config_file("gestures.gin")
model = rnn_models.BaseRNN()

Test the model. What is the output shape you need? Remember, we are doing classification!

In [37]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [38]:
from src.models import metrics

accuracy = metrics.Accuracy()
accuracy(y, yhat)

tensor(0.0625)

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [39]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [40]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(3.0288, grad_fn=<NllLossBackward0>)

In [42]:
from datetime import datetime
from pathlib import Path
gin.parse_config_file("gestures.gin")

log_dir = Path("../../models/gestures/")

model = rnn_models.BaseRNN()

model = train_model.trainloop(
    model=model,
    metrics=[accuracy],
    train_dataloader=trainloader,
    test_dataloader=testloader,
    log_dir=log_dir,
    train_steps=len(trainloader),
    eval_steps=len(testloader)
)

2022-05-23 14:55:27.701 | INFO     | src.data.data_tools:dir_add_timestamp:209 - Logging to ../../models/gestures/20220523-1455
100%|██████████| 81/81 [00:01<00:00, 48.52it/s]
2022-05-23 14:55:29.653 | INFO     | src.models.train_model:trainloop:156 - Epoch 0 train 2.4343 test 0.1240 metric ['0.1125']
100%|██████████| 81/81 [00:01<00:00, 70.77it/s]
2022-05-23 14:55:30.893 | INFO     | src.models.train_model:trainloop:156 - Epoch 1 train 2.2257 test 0.1232 metric ['0.0906']
100%|██████████| 81/81 [00:01<00:00, 73.09it/s]
2022-05-23 14:55:32.102 | INFO     | src.models.train_model:trainloop:156 - Epoch 2 train 2.2427 test 0.1152 metric ['0.1109']
100%|██████████| 81/81 [00:00<00:00, 104.80it/s]
2022-05-23 14:55:32.964 | INFO     | src.models.train_model:trainloop:156 - Epoch 3 train 2.1771 test 0.1162 metric ['0.1344']
100%|██████████| 81/81 [00:00<00:00, 105.80it/s]
2022-05-23 14:55:33.810 | INFO     | src.models.train_model:trainloop:156 - Epoch 4 train 2.1127 test 0.1158 metric ['0.12

Excercises:

- improve the RNN model
- test different things. What works? What does not?
- make a second model, where you test a GRU layer. Create an additional `.gin` file for this second model. 
- Try to add Conv1D layers.

You should be able to get above 90% accuracy with the dataset.

In [15]:
gin.parse_config_file("gestures_gru.gin")

log_dir = Path("../../models/gestures/")

model = rnn_models.GRUmodel()

model = rnn_models.trainloop(
    model=model,
    metrics=[accuracy],
    train_dataloader=trainloader,
    test_dataloader=testloader,
    log_dir=log_dir,
    train_steps=len(trainloader),
    eval_steps=len(testloader)
)